<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>

# Curso *Ingeniería de Características*

### Proyecto 1. *Descargando datos de la Web*


<p> Osiris Alejandro Izaguirre </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>

</center>

# Objetivo

Debido a la situación actual en términos de inseguridad en el estado de Sonora, así como en el resto del país, se plantea la extracción de datos públicos para el conocimiento y entendimiento de la forma en la que se presenta en los diferentes municipios y como se ven relacionadoa estos datos con el tema de **Crimen Organizado** y **Narcomenudeo** 

Para este proposito se utilizarán datos encontrados en la página oficial del [*Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública*](https://www.gob.mx/sesnsp) sobre los casos por tipo de delito por municipio. 

Se consideran además datos sobre la percepción sobre seguridad publica en cada municipio de la *Encuesta Nacional de Victimización y Percepción sobre la Seguridad Pública (ENVIPE)* recabados por el [INEGI](https://www.inegi.org.mx/default.html). 

# Importación de librerias

Comenzamos con la importación de las librerias a utilizar a lo largo del proyecto:

In [1]:
!pip install gdown
!pip install zipfile2
!pip install openpyxl
!pip install pyarrow
!pip install fastparquet


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os             # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
import datetime       # Fecha de descarga
import pandas as pd   # Solo para ver el archivo descargado
import zipfile        # Descompresión de archivos
import gdown          # Para descargar archivos grandes desde Google Drive
import numpy as np
import re

# Definiendo las rutas de los Datos a Utilizar

Es importante saber en donde nos encontramos y crear los subdirectorios necesarios para guardar los datos de manera ordenada. Tambien es importante evitar cargar datos que ya han sido descargados anteriormente.

In [3]:
# pwd
print(os.getcwd())

#  Estos son los datos que vamos a descargar y donde vamos a guardarlos
delitos_url = "https://drive.google.com/u/0/uc?id=10fm7xF4F6X1uGDRNw5aEMOJo_GsFIeTR&export=download"
delitos_archivo = "delitos.csv"
delitos_diccionario_url = "https://drive.google.com/u/0/uc?id=1rfvgcAcEzLR1Q44wwjZhjBvBFjtGSmX3&export=download"
delitos_diccionario_archivo = "delitos_diccionario.xlsx"

#  Los datos de encuestas de percepción sobre la seguridad pública se encuentran publicados por año 
#  por lo que se crean estos pedazos de URL para iterar utilizando una variable "año" al descargar
percepcion_base_url = "https://www.inegi.org.mx/contenidos/programas/envipe/"
percepcion_datosurl = "/datosabiertos/conjunto_de_datos_ENVIPE"
percepcion_datos_url = "/datosabiertos/conjunto_de_datos_ENVIPE_"
percepciondatosurl = "/datosabiertos/ENVIPE_"
percepcion_final_url = "_csv.zip"
percepcion_archivo = "_percepción.zip"

# Los datos sobre densidad poblacional del INEGI para 2020.
poblacion_url = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/iter/iter_26_cpv2020_csv.zip"
poblacion_archivo = "poblacion2020.zip"

subdir = "./data/"


C:\Users\oaiza\OneDrive\Documentos\PCD Python\pcd


# Descargando y extrayendo los Datos

Debido a que las bases de datos del INEGI desde donde se van a descargar los datos complementarios sobre percepción de la seguridad pública no se encuentran completamente estandarizadas en sus URL se realizan condiciones especiales para la descarga de cada año.

In [4]:
if not os.path.exists(subdir + delitos_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    gdown.download(delitos_url, subdir + delitos_archivo)    
    gdown.download(delitos_diccionario_url, subdir + delitos_diccionario_archivo)   

    # Se encontraron algunas pequeñas variaciones en los URL de descarga de los diferentes años:
    for year in range(2017, 2023):
    
      if(year == 2018 or year == 2019 or year == 2020):
        gdown.download(percepcion_base_url + str(year) + percepcion_datosurl + str(year) + percepcion_final_url, 
                                 subdir + str(year) + percepcion_archivo)
        with zipfile.ZipFile(subdir + str(year) + percepcion_archivo, "r") as zip_ref:
          zip_ref.extractall(subdir)   
    
      elif(year == 2017):
        gdown.download(percepcion_base_url + str(year) + percepciondatosurl + str(year) + percepcion_final_url, 
                                 subdir + str(year) + percepcion_archivo)
        with zipfile.ZipFile(subdir + str(year) + percepcion_archivo, "r") as zip_ref:
          zip_ref.extractall(subdir) 
      
      else:
        gdown.download(percepcion_base_url + str(year) + percepcion_datos_url + str(year) + percepcion_final_url, 
                                 subdir + str(year) + percepcion_archivo)
        with zipfile.ZipFile(subdir + str(year) + percepcion_archivo, "r") as zip_ref:
          zip_ref.extractall(subdir) 
        
    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre delincuencia en Sonora\n")
        info = """
        Datos sobre los delitos denunciados oficialmente en la página del Secretariado Ejecutivo del Sistema Nacional 
        de Seguridad Pública, complementados con datos del INEGI sobre la percepción sobre seguridad pública en Sonora.             

        Los datos se obtuvieron del SESNSP con fecha de 20 de septiembre de 2022 (la base de datos se actualiza constantemente) 

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + delitos_url + "\n")
        f.write("Nombre: " + delitos_archivo + "\n")
        
        for year in range(2017, 2023):
          
          if(year == 2018 or year == 2019 or year == 2020):
            f.write("Agregados sobre percepción nacional descargados desde: " + percepcion_base_url + str(year) + percepcion_datosurl + str(year) + percepcion_final_url + "\n")
            f.write("Nombre: " + str(year) + percepcion_archivo + "\n")
        
          elif(year == 2017):
            f.write("Agregados sobre percepción nacional descargados desde: " + percepcion_base_url + str(year) + percepciondatosurl + str(year) + percepcion_final_url + "\n")
            f.write("Nombre: " + str(year) + percepcion_archivo + "\n")
          
          else:
            f.write("Agregados sobre percepción nacional descargados desde: " + percepcion_base_url + str(year) + percepcion_datos_url + str(year) + percepcion_final_url + "\n")
            f.write("Nombre: " + str(year) + percepcion_archivo + "\n")


In [5]:
gdown.download(poblacion_url, subdir + poblacion_archivo)
with zipfile.ZipFile(subdir + poblacion_archivo, "r") as zip_ref:
    zip_ref.extractall(subdir)

Downloading...
From: https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/iter/iter_26_cpv2020_csv.zip
To: C:\Users\oaiza\OneDrive\Documentos\PCD Python\pcd\data\poblacion2020.zip
100%|███████████████████████████████████████████████████████████████████████████████| 799k/799k [00:00<00:00, 2.44MB/s]


# Leyendo los archivos .csv en Data Frames de Pandas

Es necesario leer los archivos .csv e importarlos como un data frame utilizable con las librerias de pandas y otras en python para su procesamiento. 

In [6]:
# Archivo sobre Delitos
delitos_df = pd.read_csv(subdir + delitos_archivo, encoding = 'latin-1')

# Se importaran los DataFrames de manera individual devido a la falta de estandarización del INEGI

# Algunos de los archivos no se pudieron leer con codificación utf-8 por lo que fue necesario especificar 
# el tipo de codificación 'latin-1'

# Archivo sobre Percepción de Seguridad Pública 2017
percepcion2017_df = pd.read_csv(subdir + 'tper_vic1_envipe2017/conjunto_de_datos/tper_vic1.csv',
                                encoding = 'latin-1')

# Archivo sobre Percepción de Seguridad Pública 2018
percepcion2018_df = pd.read_csv(subdir + 'conjunto_de_datos_tper_vic1_envipe_2018/conjunto_de_datos/conjunto_de_datos_tper_vic1_envipe_2018.csv', 
                                encoding = 'utf-8-sig')

# Archivo sobre Percepción de Seguridad Pública 2019
percepcion2019_df = pd.read_csv(subdir + 'conjunto_de_datos_TPer_Vic1_ENVIPE_2019/conjunto_de_datos/conjunto_de_datos_TPer_Vic1_ENVIPE_2019.csv',
                               encoding = 'utf-8')

# Archivo sobre Percepción de Seguridad Pública 2020
percepcion2020_df = pd.read_csv(subdir + 'conjunto_de_datos_TPer_Vic1_ENVIPE_2020/conjunto_de_datos/conjunto_de_datos_TPer_Vic1_ENVIPE_2020.csv', 
                                encoding = 'latin-1')

# Archivo sobre Percepción de Seguridad Pública 2021
percepcion2021_df = pd.read_csv(subdir + 'conjunto_de_datos_TPer_Vic1_ENVIPE_2021/conjunto_de_datos/conjunto_de_datos_TPer_Vic1_ENVIPE_2021.csv', 
                                encoding = 'latin-1')

#Archivo sobre Percepción de Seguridad Pública 2022
percepcion2022_df = pd.read_csv(subdir + 'conjunto_de_datos_TPer_Vic1_ENVIPE_2022/conjunto_de_datos/conjunto_de_datos_TPer_Vic1_ENVIPE_2022.csv', 
                                encoding = 'latin-1')

#Archivos con densidad poblacional en Sonora
poblacion_df = pd.read_csv(subdir + 'iter_26_cpv2020\conjunto_de_datos\conjunto_de_datos_iter_26CSV20.csv')

# Generando los diccionarios de datos como archivos .csv

Se leen de las diferentes fuentes los dos diccinarios de datos de las tablas a utilizar. De la previsualización se encontró que el primer archivo sobre los datos del SESNSP sobre delitos se encuentra capturado en un archivo .xlsx con diferentes pestañas (sobre diferentes datos que no se utilizan en este analisis) además de encabezados con descripciones y/o nombres de las tablas que no son utiles y no se pueden leer como Data Frame de pandas, por lo que se tiene que leer de manera especial. 

Para los datos del INEGI sobre percepción de la seguridad pública se cuenta con un diccionario por cada conjunto de datos, todos estos continen la misma información al ser una encuesta estandarizada, se decidió utilizar el diccionario de datos del año 2020 por tener el mayor numero de entradas. Al visualizarlas se encontraron muchas entradas repetidas por lo que es necesario realizar una limpieza de los datos.

In [7]:
delitos_diccionario_df = pd.read_excel(subdir + delitos_diccionario_archivo, sheet_name='IDM_NM', header=1)

# Todos los archivos del INEGI vienen con sus respectivos diccionarios de datos, 
# se utilizara solamente el del año 2020 ya que todos tienen la misma información
# pero este es le año con mayor numero de entradas.
percepcion_diccionario_df = pd.read_csv(subdir + 'conjunto_de_datos_TPer_Vic1_ENVIPE_2020/diccionario_de_datos/diccionario_de_datos_TPer_Vic1_ENVIPE_2020.csv', 
                                 encoding = 'latin-1')

# Limpieza de observaciones duplicadas.
percepcion_diccionario_df.drop_duplicates(subset=["NOMBRE_CAMPO"], keep='first', inplace=True)

percepcion_diccionario_df['RANGO_CLAVES'] = percepcion_diccionario_df['RANGO_CLAVES'].str.strip()

delitos_diccionario_df.to_csv(subdir + 'delitos_diccionario.csv', index = False, encoding = 'utf-8')
percepcion_diccionario_df.to_csv(subdir + 'percepción_diccionario.csv', index = False, encoding = 'utf-8')

In [8]:
# Visualizado los datos
delitos_diccionario_df

,Campo,Descripción
0,Año,Año de registro de las averiguaciones previas ...
1,Clave_Ent,"Clave de la entidad, según el Marco Geoestadís..."
2,Entidad,Entidad federativa de registro de las averigua...
3,Cve. Municipio,"Clave del municipio, según el Marco Geoestadís..."
4,Municipio,Municipio de registro de las averiguaciones pr...
5,Bien jurídico afectado,Primera clasificación de los delitos en las av...
6,Tipo de delito,Segunda clasificación de los delitos.
7,Subtipo de delito,Tercera clasificación de los delitos.
8,Modalidad,Cuarta clasificación de los delitos.
9,Enero - diciembre,Mes de registro de las averiguaciones previas ...


In [9]:
percepcion_diccionario_df

,NOMBRE_CAMPO,NEMONICO,TIPO,LONGITUD,RANGO_CLAVES
0,Identificador de vivienda seleccionada,ID_VIV,Alfanumérico,10,0100001.01 3299999.99
1,Identificador del hogar seleccionado,ID_HOG,Alfanumérico,13,0100001.01.01 3299999.99.99
2,Identificador del informante seleccionado,ID_PER,Alfanumérico,16,0100001.01.01.01 3299999.99.99.30
3,Control de vivienda (UPM),UPM,Numérico,7,0100001 3299999
4,Vivienda seleccionada,VIV_SEL,Numérico,2,01 99
...,...,...,...,...,...
708,Factor de personas elegidas del área metropoli...,FAC_ELE_AM,Numérico,6,000001 999999
709,Dominio u ámbito,DOMINIO,Alfanumérico,1,U
712,Estrato sociodemográfico,ESTRATO,Numérico,1,1
716,Estrato de diseño muestral,EST_DIS,Numérico,3,001 303


# Revisando los datos sobre Delitos del SESNSP

Comenzamos a revisar los datos del DataFrame `delitos_df` para identificar el tipo de datos de cada columna y diferentes features significativos para hacer su limpieza respectiva y seleccionar las caracteristicas significativas para este estudio.

In [10]:
delitos_df.info()
print('\n')
delitos_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832404 entries, 0 to 1832403
Data columns (total 21 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Año                     int64  
 1   Clave_Ent               int64  
 2   Entidad                 object 
 3   Cve. Municipio          int64  
 4   Municipio               object 
 5   Bien jurídico afectado  object 
 6   Tipo de delito          object 
 7   Subtipo de delito       object 
 8   Modalidad               object 
 9   Enero                   int64  
 10  Febrero                 int64  
 11  Marzo                   int64  
 12  Abril                   int64  
 13  Mayo                    int64  
 14  Junio                   int64  
 15  Julio                   int64  
 16  Agosto                  int64  
 17  Septiembre              float64
 18  Octubre                 float64
 19  Noviembre               float64
 20  Diciembre               float64
dtypes: float64(4), int64(11), objec

Año                            0
Clave_Ent                      0
Entidad                        0
Cve. Municipio                 0
Municipio                      0
Bien jurídico afectado         0
Tipo de delito                 0
Subtipo de delito              0
Modalidad                      0
Enero                          0
Febrero                        0
Marzo                          0
Abril                          0
Mayo                           0
Junio                          0
Julio                          0
Agosto                         0
Septiembre                242942
Octubre                   242942
Noviembre                 242942
Diciembre                 242942
dtype: int64

In [11]:
# Convertimos todas las columnas con variables categoricas a formato str
delitos_df['Entidad'] = delitos_df['Entidad'].astype('string')
delitos_df['Municipio'] = delitos_df['Municipio'].astype('string')
delitos_df['Bien jurídico afectado'] = delitos_df['Bien jurídico afectado'].astype('string')
delitos_df['Tipo de delito'] = delitos_df['Tipo de delito'].astype('string')
delitos_df['Subtipo de delito'] = delitos_df['Subtipo de delito'].astype('string')
delitos_df['Modalidad'] = delitos_df['Modalidad'].astype('string')

# Convertimos los datos faltantes en 0
delitos_df['Septiembre'] = delitos_df['Septiembre'].fillna(0)
delitos_df['Octubre'] = delitos_df['Octubre'].fillna(0)
delitos_df['Noviembre'] = delitos_df['Noviembre'].fillna(0)
delitos_df['Diciembre'] = delitos_df['Diciembre'].fillna(0)

# Convertimos todas las columnas con variables de tipo flotante a enteros (ya que no 
# tiene sentido físico el tener una carpeta de investigación o denuncia fraccionada)
delitos_df['Septiembre'] = delitos_df['Septiembre'].astype(int)
delitos_df['Octubre'] = delitos_df['Octubre'].astype(int)
delitos_df['Noviembre'] = delitos_df['Noviembre'].astype(int)
delitos_df['Diciembre'] = delitos_df['Diciembre'].astype(int)

#delitos_df
delitos_df.dtypes

Año                        int64
Clave_Ent                  int64
Entidad                   string
Cve. Municipio             int64
Municipio                 string
Bien jurídico afectado    string
Tipo de delito            string
Subtipo de delito         string
Modalidad                 string
Enero                      int64
Febrero                    int64
Marzo                      int64
Abril                      int64
Mayo                       int64
Junio                      int64
Julio                      int64
Agosto                     int64
Septiembre                 int32
Octubre                    int32
Noviembre                  int32
Diciembre                  int32
dtype: object

In [12]:
# Filtramos solo los valores pertenecientes al estado de Sonora.
delitos_df = delitos_df.loc[delitos_df['Entidad'] == 'Sonora']
delitos_df = delitos_df.loc[delitos_df['Año'] >= 2017]

# Borramos las caracteristicas inecesarias para este caso de estudio

# Se borraran las columnas 'Clave_Ent' y 'Entidad' ya que solo tenemos observaciones del estado de Sonora
del delitos_df['Clave_Ent']
del delitos_df['Entidad']

# Se borraran las columnas 'Bien jurídico afectado', 'Subtipo de delito' y 'Modalidad' ya que para el caso
# de estudio nos interesa solamente el tipo de delito cometido.
del delitos_df['Bien jurídico afectado']
del delitos_df['Subtipo de delito']
del delitos_df['Modalidad']

# Agregamos la columa 'Total' para sumarizar los casos denunciados por año 
delitos_df['Total'] = (delitos_df['Enero'] + delitos_df['Febrero'] + delitos_df['Marzo'] +
                       delitos_df['Abril'] + delitos_df['Mayo'] + delitos_df['Junio'] + 
                       delitos_df['Julio'] + delitos_df['Agosto'] + delitos_df['Septiembre'] +
                       delitos_df['Octubre'] + delitos_df['Noviembre'] + delitos_df['Diciembre']) 

delitos_df

,Año,Cve. Municipio,Municipio,Tipo de delito,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Total
564088,2017,26001,Aconchi,Homicidio,0,0,0,0,0,0,0,0,0,0,0,0,0
564089,2017,26001,Aconchi,Homicidio,0,0,0,0,0,0,0,0,0,0,0,0,0
564090,2017,26001,Aconchi,Homicidio,0,0,0,0,0,0,0,0,0,0,0,0,0
564091,2017,26001,Aconchi,Homicidio,0,0,0,0,0,0,0,0,0,0,0,0,0
564092,2017,26001,Aconchi,Homicidio,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783595,2022,26999,Otros Municipios,Falsificación,0,0,0,0,0,0,0,0,0,0,0,0,0
1783596,2022,26999,Otros Municipios,Contra el medio ambiente,0,0,0,0,0,0,0,0,0,0,0,0,0
1783597,2022,26999,Otros Municipios,Delitos cometidos por servidores públicos,0,0,0,0,0,0,0,0,0,0,0,0,0
1783598,2022,26999,Otros Municipios,Electorales,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
delitos_df = delitos_df.groupby(['Año', 'Municipio', 'Tipo de delito']).agg({'Total':['sum']})
delitos_df.columns = delitos_df.columns.to_flat_index()
delitos_df = delitos_df.reset_index()
delitos_df.columns = ['Año', 'Municipio', 'Tipo de delito', 'Total']
delitos_df

,Año,Municipio,Tipo de delito,Total
0,2017,Aconchi,Aborto,0
1,2017,Aconchi,Abuso de confianza,1
2,2017,Aconchi,Abuso sexual,0
3,2017,Aconchi,Acoso sexual,0
4,2017,Aconchi,Allanamiento de morada,2
...,...,...,...,...
17755,2022,Yécora,Tráfico de menores,0
17756,2022,Yécora,Violación equiparada,0
17757,2022,Yécora,Violación simple,1
17758,2022,Yécora,Violencia de género en todas sus modalidades d...,0


In [14]:
delitos_df = pd.pivot(delitos_df, index=['Año', 'Municipio'], columns='Tipo de delito', values='Total')
delitos_df.columns = delitos_df.columns.to_flat_index()
delitos_df = delitos_df.reset_index()
delitos_df

Tipo de delito,Año,Municipio,Aborto,Abuso de confianza,Abuso sexual,Acoso sexual,Allanamiento de morada,Amenazas,Contra el medio ambiente,Corrupción de menores,...,Otros delitos que atentan contra la vida y la ...,Rapto,Robo,Secuestro,Trata de personas,Tráfico de menores,Violación equiparada,Violación simple,Violencia de género en todas sus modalidades d...,Violencia familiar
0,2017,Aconchi,0,1,0,0,2,1,0,0,...,0,0,7,0,0,1,0,0,0,1
1,2017,Agua Prieta,0,4,11,0,0,4,0,1,...,0,0,124,0,0,2,6,6,0,17
2,2017,Alamos,0,0,0,0,0,1,0,0,...,0,0,29,0,0,0,1,1,0,0
3,2017,Altar,0,0,1,1,0,3,0,0,...,0,0,8,0,0,0,0,2,0,1
4,2017,Arivechi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,2022,Tubutama,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
440,2022,Ures,0,0,2,0,2,0,0,0,...,0,0,4,0,0,0,0,0,0,1
441,2022,Villa Hidalgo,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,1
442,2022,Villa Pesqueira,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Eliminar caracteristicas no significativas
delitos_df.columns

del delitos_df['Aborto']
del delitos_df['Abuso de confianza']
del delitos_df['Acoso sexual']
del delitos_df['Allanamiento de morada']
del delitos_df['Amenazas']
del delitos_df['Contra el medio ambiente']
del delitos_df['Corrupción de menores']
del delitos_df['Daño a la propiedad']
del delitos_df['Delitos cometidos por servidores públicos']
del delitos_df['Despojo']
del delitos_df['Electorales']
del delitos_df['Evasión de presos']
del delitos_df['Extorsión']
del delitos_df['Falsedad']
del delitos_df['Falsificación']
del delitos_df['Fraude']
del delitos_df['Hostigamiento sexual']
del delitos_df['Incesto']
del delitos_df['Incumplimiento de obligaciones de asistencia familiar']
del delitos_df['Lesiones']
del delitos_df['Otros delitos contra el patrimonio']
del delitos_df['Otros delitos contra la familia']
del delitos_df['Otros delitos contra la sociedad']
del delitos_df['Otros delitos del Fuero Común']
del delitos_df['Otros delitos que atentan contra la libertad personal']
del delitos_df['Otros delitos que atentan contra la libertad y la seguridad sexual']
del delitos_df['Otros delitos que atentan contra la vida y la integridad corporal']
del delitos_df['Rapto']
del delitos_df['Trata de personas']
del delitos_df['Tráfico de menores']
del delitos_df['Violación equiparada']
del delitos_df['Violación simple']
del delitos_df['Violencia de género en todas sus modalidades distinta a la violencia familiar']
del delitos_df['Violencia familiar']

delitos_df

Tipo de delito,Año,Municipio,Abuso sexual,Feminicidio,Homicidio,Narcomenudeo,Robo,Secuestro
0,2017,Aconchi,0,0,2,1,7,0
1,2017,Agua Prieta,11,0,18,9,124,0
2,2017,Alamos,0,1,11,0,29,0
3,2017,Altar,1,0,3,0,8,0
4,2017,Arivechi,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
439,2022,Tubutama,0,0,1,0,3,0
440,2022,Ures,2,0,2,2,4,0
441,2022,Villa Hidalgo,0,0,0,0,1,0
442,2022,Villa Pesqueira,0,0,1,0,0,0


In [16]:
delitos_df.Municipio.unique()

<StringArray>
[                      'Aconchi',                   'Agua Prieta',
                        'Alamos',                         'Altar',
                      'Arivechi',                        'Arizpe',
                          'Atil',                  'Bacadéhuachi',
                      'Bacanora',                       'Bacerac',
                      'Bacoachi',                     'Banámichi',
                       'Bavispe',                     'Baviácora',
                 'Benito Juárez',                 'Benjamín Hill',
                         'Bácum',                       'Caborca',
                        'Cajeme',                       'Cananea',
                         'Carbó',                       'Cucurpe',
                        'Cumpas',                   'Divisaderos',
                       'Empalme',                      'Etchojoa',
                     'Fronteras', 'General Plutarco Elías Calles',
                      'Granados',               

In [17]:
delitos_df.drop(delitos_df[delitos_df['Municipio'].isin(['No Especificado', 'Otros Municipios'])].index, inplace = True)
delitos_df.Municipio.unique()

<StringArray>
[                      'Aconchi',                   'Agua Prieta',
                        'Alamos',                         'Altar',
                      'Arivechi',                        'Arizpe',
                          'Atil',                  'Bacadéhuachi',
                      'Bacanora',                       'Bacerac',
                      'Bacoachi',                     'Banámichi',
                       'Bavispe',                     'Baviácora',
                 'Benito Juárez',                 'Benjamín Hill',
                         'Bácum',                       'Caborca',
                        'Cajeme',                       'Cananea',
                         'Carbó',                       'Cucurpe',
                        'Cumpas',                   'Divisaderos',
                       'Empalme',                      'Etchojoa',
                     'Fronteras', 'General Plutarco Elías Calles',
                      'Granados',               

In [18]:
delitos_df.columns

Index(['Año', 'Municipio', 'Abuso sexual', 'Feminicidio', 'Homicidio',
       'Narcomenudeo', 'Robo', 'Secuestro'],
      dtype='string', name='Tipo de delito')

In [19]:
# Guardamos el Data Frame 'delitos_df' de pandas como .parquet
delitos_df.to_parquet(subdir + 'delitos_df.parquet')

# Revisando los datos sobre Percepción de la Seguridad Pública del INEGI

Comenzamos a revisar los datos de los DataFrame `percepcionXXXX_df` y concatenar todos los datos en un solo DataFrame agrupados por la variable 'Año' para poderlo relacionar con el DataFrame `delitos_df`. Además, identificar el tipo de datos de cada columna y diferentes features significativos para hacer su limpieza respectiva y seleccionar las caracteristicas significativas para este estudio. 

In [20]:
percepcion2017_df.info()
print('\n')
percepcion2018_df.info()
print('\n')
percepcion2019_df.info()
print('\n')
percepcion2020_df.info()
print('\n')
percepcion2021_df.info()
print('\n')
percepcion2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92551 entries, 0 to 92550
Columns: 186 entries, ID_VIV to UPM_DIS
dtypes: float64(1), int64(125), object(60)
memory usage: 131.3+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91541 entries, 0 to 91540
Columns: 187 entries, ID_VIV to llave
dtypes: float64(2), int64(125), object(60)
memory usage: 130.6+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91896 entries, 0 to 91895
Columns: 186 entries, ID_VIV to UPM_DIS
dtypes: float64(1), int64(125), object(60)
memory usage: 130.4+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90571 entries, 0 to 90570
Columns: 184 entries, ID_VIV to UPM_DIS
dtypes: float64(1), int64(130), object(53)
memory usage: 127.1+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92051 entries, 0 to 92050
Columns: 180 entries, ID_VIV to UPM_DIS
dtypes: float64(1), int64(130), object(49)
memory usage: 126.4+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92103 entries, 0 to 92102
C

In [21]:
percepcion2017_df.NOM_ENT.unique()

array(['Aguascalientes\r', 'Baja California\r', 'Baja California Sur\r',
       'Campeche\r', 'Coahuila de Zaragoza\r', 'Colima\r', 'Chiapas\r',
       'Chihuahua\r', 'Ciudad de MÃ\x83Â©xico\r', 'Durango\r',
       'Guanajuato\r', 'Guerrero\r', 'Hidalgo\r', 'Jalisco\r',
       'MÃ\x83Â©xico\r', 'MichoacÃ\x83Â¡n de Ocampo\r', 'Morelos\r',
       'Nayarit\r', 'Nuevo LeÃ\x83Â³n\r', 'Oaxaca\r', 'Puebla\r',
       'QuerÃ\x83Â©taro\r', 'Quintana Roo\r',
       'San Luis PotosÃ\x83Â\xad\r', 'Sinaloa\r', 'Sonora\r', 'Tabasco\r',
       'Tamaulipas\r', 'Tlaxcala\r', 'Veracruz de Ignacio de la Llave\r',
       'YucatÃ\x83Â¡n\r', 'Zacatecas\r'], dtype=object)

In [22]:
# Agregamos la columna 'Año' en cada DataFrame para poderlos concatenar y comparar contra el de delitos
percepcion2017_df.insert(0, 'Año', 2017, False)
percepcion2018_df.insert(0, 'Año', 2018, False)
percepcion2019_df.insert(0, 'Año', 2019, False)
percepcion2020_df.insert(0, 'Año', 2020, False)
percepcion2021_df.insert(0, 'Año', 2021, False)
percepcion2022_df.insert(0, 'Año', 2022, False)

# Concatenamos todos los DataFrames anuales en uno solo
percepcion_df = pd.concat([percepcion2017_df, percepcion2018_df,
                           percepcion2019_df, percepcion2020_df,
                           percepcion2021_df, percepcion2022_df], axis=0, ignore_index = True)

percepcion_df

,Año,ID_VIV,ID_HOG,ID_PER,UPM,VIV_SEL,HOGAR,RESUL_H,R_SEL,SEXO,...,llave,AP4_2_13,AP4_5_17,AP4_5_18,AP4_10_16,AP5_3_11,AP5_4_11,AP5_5_11,AP5_6_11,AP4_4_A
0,2017,100008.01,0100008.01.01\r,0100008.01.01.01\r,100008,1,1,B\r,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,100008.02,0100008.02.01\r,0100008.02.01.02\r,100008,2,1,B\r,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,100008.04,0100008.04.01\r,0100008.04.01.01\r,100008,4,1,B\r,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,100008.05,0100008.05.01\r,0100008.05.01.01\r,100008,5,1,A\r,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,100022.01,0100022.01.01\r,0100022.01.01.03\r,100022,1,1,A\r,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550708,2022,3260721.11,3260721.11.01\r,3260721.11.01.03\r,3260721,11,1,B\r,3,1,...,NaN,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,2.0
550709,2022,3260721.12,3260721.12.01\r,3260721.12.01.01\r,3260721,12,1,B\r,1,1,...,NaN,0.0,0.0,1.0,3.0,NaN,NaN,NaN,NaN,3.0
550710,2022,3260721.13,3260721.13.01\r,3260721.13.01.02\r,3260721,13,1,B\r,2,2,...,NaN,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,4.0
550711,2022,3260721.14,3260721.14.01\r,3260721.14.01.02\r,3260721,14,1,B\r,2,2,...,NaN,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,3.0


In [23]:
for col in percepcion_df.columns:
  percepcion_df[col] = percepcion_df[col].fillna(0)

  # Convertimos todos las columnas de tipo object a string y limpiamos errores de lectura como '\r'
  if(percepcion_df[col].dtypes == 'object'): 
    percepcion_df[col] = percepcion_df[col].str.strip().astype('string')

  # Convertimos observaciones faltantes NaN a 0 y cambiamos el tipo de datos a int
  if(percepcion_df[col].dtypes == 'float64'):
    percepcion_df[col] = percepcion_df[col].fillna(0)
    percepcion_df[col] = percepcion_df[col].astype(int)

percepcion_df.dtypes

Año          int64
ID_VIV       int32
ID_HOG      string
ID_PER      string
UPM          int64
             ...  
AP5_3_11     int32
AP5_4_11    string
AP5_5_11    string
AP5_6_11    string
AP4_4_A      int32
Length: 197, dtype: object

In [24]:
# Filtramos solo los valores pertenecientes al estado de Sonora.
percepcion_df = percepcion_df.loc[percepcion_df['NOM_ENT'] == 'Sonora']

# Borramos las caracteristicas inecesarias para este caso de estudio

# Se borraran las columnas 'CVE_ENT' y 'NOM_ENT' ya que solo tenemos observaciones del estado de Sonora
del percepcion_df['NOM_ENT']
del percepcion_df['CVE_ENT']

# Se borraran las columnas inecesarias 
del percepcion_df['ID_VIV']
del percepcion_df['ID_HOG']
del percepcion_df['ID_PER']
del percepcion_df['UPM']
del percepcion_df['VIV_SEL']
del percepcion_df['HOGAR']
del percepcion_df['RESUL_H']
del percepcion_df['R_SEL']
del percepcion_df['AREAM']
del percepcion_df['FAC_HOG']
del percepcion_df['FAC_ELE']
del percepcion_df['FAC_HOG_AM']
del percepcion_df['FAC_ELE_AM']
del percepcion_df['DOMINIO']
del percepcion_df['EST_DIS']
del percepcion_df['UPM_DIS']
del percepcion_df['llave']

del percepcion_df['AP4_1'] # Tiempo en vivienda
del percepcion_df['AP4_2_01'] # Pobreza
del percepcion_df['AP4_2_02'] # Desempleo
del percepcion_df['AP4_2_04'] # Aumento de precios
del percepcion_df['AP4_2_06'] # Desastres Naturales
del percepcion_df['AP4_2_07'] # Escaces de agua
del percepcion_df['AP4_2_08'] # Corrupcion
del percepcion_df['AP4_2_09'] # Educacion
del percepcion_df['AP4_2_10'] # Salud
del percepcion_df['AP4_2_11'] # Falta de castigo
del percepcion_df['AP4_2_12'] # 
del percepcion_df['AP4_2_13'] # 
del percepcion_df['AP4_2_99'] # 

del percepcion_df['AP4_4_01'] # En casa
del percepcion_df['AP4_4_02'] # Trabajo
del percepcion_df['AP4_4_04'] # Escuela
del percepcion_df['AP4_4_05'] # Mercado
del percepcion_df['AP4_4_06'] # Centro Comercial
del percepcion_df['AP4_4_07'] # Banco
del percepcion_df['AP4_4_08'] # Cajero
del percepcion_df['AP4_4_09'] # Transporte
del percepcion_df['AP4_4_10'] # Automovil
del percepcion_df['AP4_4_11'] # Carretera

del percepcion_df['AP4_5_01'] # Consumo de alcohol
del percepcion_df['AP4_5_02'] # Pandillerismo
del percepcion_df['AP4_5_03'] # Riñas entre vecinos
del percepcion_df['AP4_5_04'] # Pirateria
del percepcion_df['AP4_5_05'] # Venta ilegal
del percepcion_df['AP4_5_06'] # Violencia policiaca
del percepcion_df['AP4_5_07'] # Invasion
del percepcion_df['AP4_5_08'] # Consumo de droga
del percepcion_df['AP4_5_12'] # Prostitucion
del percepcion_df['AP4_5_15'] # Extorsion
del percepcion_df['AP4_5_16'] # Huachicol
del percepcion_df['AP4_5_17'] # Diablitos
del percepcion_df['AP4_5_18'] # 
del percepcion_df['AP4_5_99'] # 

del percepcion_df['AP4_6_1'] # 
del percepcion_df['AP4_6_2'] # 
del percepcion_df['AP4_6_3'] # 

del percepcion_df['AP4_7_1'] # 
del percepcion_df['AP4_7_2'] # 
del percepcion_df['AP4_7_3'] # 

del percepcion_df['AP4_8_1'] # 
del percepcion_df['AP4_9_1'] # 
del percepcion_df['AP4_8_2'] # 
del percepcion_df['AP4_9_2'] # 
del percepcion_df['AP4_8_3'] # 
del percepcion_df['AP4_9_3'] # 
del percepcion_df['AP4_8_4'] # 
del percepcion_df['AP4_9_4'] # 
del percepcion_df['AP4_8_5'] # 
del percepcion_df['AP4_9_5'] # 
del percepcion_df['AP4_8_6'] # 
del percepcion_df['AP4_9_6'] # 

del percepcion_df['AP4_10_03'] # Visitar parientes
del percepcion_df['AP4_10_04'] # Taxi
del percepcion_df['AP4_10_05'] # Transporte
del percepcion_df['AP4_10_06'] # Llevar dinero en efectivo 
del percepcion_df['AP4_10_07'] # Ir a la escuela
del percepcion_df['AP4_10_08'] # Cine/Teatro
del percepcion_df['AP4_10_10'] # Joyas
del percepcion_df['AP4_10_11'] # Comer
del percepcion_df['AP4_10_12'] # Tarjetas
del percepcion_df['AP4_10_13'] # Estadio
del percepcion_df['AP4_10_14'] # Centros comerciales
del percepcion_df['AP4_10_15'] # Viajar carreteras
del percepcion_df['AP4_10_16'] # Celulares

del percepcion_df['AP4_11_01'] # 
del percepcion_df['AP4_11_02'] # 
del percepcion_df['AP4_11_03'] # 
del percepcion_df['AP4_11_04'] # 
del percepcion_df['AP4_11_05'] # 
del percepcion_df['AP4_11_06'] # 
del percepcion_df['AP4_11_07'] # 
del percepcion_df['AP4_11_08'] # 
del percepcion_df['AP4_11_09'] # 
del percepcion_df['AP4_11_10'] # 
del percepcion_df['AP4_11_11'] # 
del percepcion_df['AP4_12'] # 

del percepcion_df['AP5_1_01'] # 
del percepcion_df['AP5_1_02'] # 
del percepcion_df['AP5_1_03'] # 
del percepcion_df['AP5_1_04'] # 
del percepcion_df['AP5_1_05'] # 
del percepcion_df['AP5_1_06'] # 
del percepcion_df['AP5_1_07'] # 
del percepcion_df['AP5_1_08'] # 
del percepcion_df['AP5_1_09'] # 
del percepcion_df['AP5_1_10'] # 
del percepcion_df['AP5_1_11'] # 
del percepcion_df['AP5_1_12'] # 
del percepcion_df['AP5_1_13'] # 

del percepcion_df['AP5_2_1'] # Vecinos
del percepcion_df['AP5_2_2'] # Compañeros
del percepcion_df['AP5_2_3'] # Familiares
del percepcion_df['AP5_2_4'] # Amigos

del percepcion_df['AP5_3_01'] # Transito 
del percepcion_df['AP5_5_01'] # 
del percepcion_df['AP5_6_01'] # 
del percepcion_df['AP5_7_1'] # 
del percepcion_df['AP5_3_02'] # Municipal 
del percepcion_df['AP5_5_02'] # 
del percepcion_df['AP5_6_02'] # 
del percepcion_df['AP5_7_2'] # 
del percepcion_df['AP5_3_03'] # Estatal 
del percepcion_df['AP5_5_03'] # 
del percepcion_df['AP5_6_03'] # 
del percepcion_df['AP5_7_3'] # 
del percepcion_df['AP5_3_04'] # Guardia Nacional 
del percepcion_df['AP5_5_04'] # 
del percepcion_df['AP5_6_04'] # 
del percepcion_df['AP5_7_4'] # 
del percepcion_df['AP5_3_05'] # Policia Ministerial
del percepcion_df['AP5_5_05'] # 
del percepcion_df['AP5_6_05'] # 
del percepcion_df['AP5_3_06'] # Ministerio Público
del percepcion_df['AP5_5_06'] # 
del percepcion_df['AP5_6_06'] #
del percepcion_df['AP5_3_07'] # Fiscalia Gral Rep
del percepcion_df['AP5_5_07'] # 
del percepcion_df['AP5_6_07'] # 
del percepcion_df['AP5_3_08'] # Ejercito
del percepcion_df['AP5_5_08'] # 
del percepcion_df['AP5_6_08'] #
del percepcion_df['AP5_3_09'] # Marina
del percepcion_df['AP5_5_09'] # 
del percepcion_df['AP5_6_09'] #
del percepcion_df['AP5_3_10'] # Jueces
del percepcion_df['AP5_5_10'] # 
del percepcion_df['AP5_6_10'] #
del percepcion_df['AP5_8'] # Carceles y rec

del percepcion_df['CVE_MUN']
del percepcion_df['AP5_4_01_C']
del percepcion_df['AP5_4_02_C']
del percepcion_df['AP5_4_03_C']
del percepcion_df['AP5_4_04_C']
del percepcion_df['AP5_4_05_C']
del percepcion_df['AP5_4_06_C']
del percepcion_df['AP5_4_07_C']
del percepcion_df['AP5_4_08_C']
del percepcion_df['AP5_4_09_C']
del percepcion_df['AP5_4_10_C']

del percepcion_df['AP5_3_11']
del percepcion_df['AP5_4_11']
del percepcion_df['AP5_5_11']
del percepcion_df['AP5_6_11']

percepcion_df

,Año,SEXO,EDAD,NOM_MUN,AP4_2_03,AP4_2_05,AP4_3_1,AP4_3_2,AP4_3_3,AP4_4_03,...,AP5_4_03,AP5_4_04,AP5_4_05,AP5_4_06,AP5_4_07,AP5_4_08,AP5_4_09,AP5_4_10,ESTRATO,AP4_4_A
72941,2017,2,25,Cajeme,0,1,1,2,2,2,...,4,4,4,4,4,2,2,4,3,0
72942,2017,1,43,Cajeme,0,1,1,1,1,2,...,3,3,,2,,2,1,2,3,0
72943,2017,2,51,Cajeme,0,1,2,2,2,2,...,3,3,,,,2,2,,3,0
72944,2017,1,30,Cajeme,0,1,2,2,2,2,...,2,2,,,2,2,1,,3,0
72945,2017,2,81,Cajeme,0,1,2,2,1,2,...,,,,,,,,,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532747,2022,1,80,Yécora,1,0,1,1,9,3,...,,,,,,,,,1,5
532748,2022,1,62,Yécora,0,0,1,1,1,1,...,2,2,3,2,2,1,2,2,1,1
532749,2022,2,53,Yécora,1,1,2,2,2,1,...,2,,2,,,2,2,,1,5
532750,2022,2,53,Yécora,0,1,2,1,2,2,...,,,2,,,3,2,,1,3


In [25]:
percepcion_df = percepcion_df.replace(to_replace ='San Luis RÃ\x83Â\xado Colorado', value = 'San Luis Río Colorado', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='San Luis Rio Colorado', value = 'San Luis Río Colorado', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='Puerto PeÃ\x83Â±asco', value = 'Puerto Peñasco', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='YÃ\x83Â©cora', value = 'Yécora', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='Benito JuÃ\x83Â¡rez', value = 'Benito Juárez', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='Benito Juarez', value = 'Benito Juárez', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='BÃ\x83Â¡cum', value = 'Bácum', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='Bacum', value = 'Bácum', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='BÃ\x83Â¡cum', value = 'Bácum', regex = True)
percepcion_df = percepcion_df.replace(to_replace ='Saric', value = 'Sáric', regex = True)
percepcion_df.NOM_MUN.unique()

<StringArray>
[                'Cajeme',                'Guaymas',             'Hermosillo',
                'Navojoa',                'Nogales',  'San Luis Río Colorado',
            'Agua Prieta',                'Caborca',                'Cananea',
                'Empalme',             'Huatabampo',         'Puerto Peñasco',
                  'Altar',                 'Imuris',                   'Naco',
              'Sahuaripa',                 'Yécora',          'Benito Juárez',
                  'Bácum',              'Magdalena',               'Etchojoa',
               'Huásabas',              'Moctezuma',              'Santa Ana',
                   'Ures',                'Bacerac',                'Mazatan',
               'Quiriego',                  'Sáric',               'Bacanora',
 'San Ignacio Río Muerto',     'Nacozari de García']
Length: 32, dtype: string

In [26]:
# NO CORRER ESTA CELDA
#for col in percepcion_df.columns:
#    percepcion_df[col][percepcion_df[col].isin(['NA', '']) == True] = pd.NA

In [27]:
# NO CORRER ESTA CELDA
#for col in percepcion_df.columns:
#  if percepcion_df[col].isna().sum() > 0:
#    print(col)
#    print(percepcion_df[col].isna().sum())
#    print(percepcion_df[col].dtype)
#    percepcion_df[col] = percepcion_df[col].fillna('0')
#    percepcion_df[col] = percepcion_df[col].replace('', '0', regex=True)
#    percepcion_df[col] = percepcion_df[col].replace('NA', '0', regex=True)
#    #percepcion_df[col] = percepcion_df[col].astype(int)
#    print(percepcion_df[col].dtype)
#    print('\n')

In [28]:
# NO CORRER ESTA CELDA
#for col in percepcion_df.columns:
#  if re.match('^AP\d_\d*', col):
#    percepcion_df[col] = percepcion_df[col].replace('NA', '0', regex=True)
#    percepcion_df[col] = percepcion_df[col].replace('', '0', regex=True)
#    percepcion_df[col] = percepcion_df[col].astype(int)

In [29]:
# NO CORRER ESTA CELDA
# for col in percepcion_df.columns:
#  if percepcion_df[col].isna().sum() > 0:
#    print(col)
#    print(percepcion_df[col].isna().sum())
#    print(percepcion_df[col].dtype)
#    print('\n')

In [30]:
percepcion_df

,Año,SEXO,EDAD,NOM_MUN,AP4_2_03,AP4_2_05,AP4_3_1,AP4_3_2,AP4_3_3,AP4_4_03,...,AP5_4_03,AP5_4_04,AP5_4_05,AP5_4_06,AP5_4_07,AP5_4_08,AP5_4_09,AP5_4_10,ESTRATO,AP4_4_A
72941,2017,2,25,Cajeme,0,1,1,2,2,2,...,4,4,4,4,4,2,2,4,3,0
72942,2017,1,43,Cajeme,0,1,1,1,1,2,...,3,3,,2,,2,1,2,3,0
72943,2017,2,51,Cajeme,0,1,2,2,2,2,...,3,3,,,,2,2,,3,0
72944,2017,1,30,Cajeme,0,1,2,2,2,2,...,2,2,,,2,2,1,,3,0
72945,2017,2,81,Cajeme,0,1,2,2,1,2,...,,,,,,,,,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532747,2022,1,80,Yécora,1,0,1,1,9,3,...,,,,,,,,,1,5
532748,2022,1,62,Yécora,0,0,1,1,1,1,...,2,2,3,2,2,1,2,2,1,1
532749,2022,2,53,Yécora,1,1,2,2,2,1,...,2,,2,,,2,2,,1,5
532750,2022,2,53,Yécora,0,1,2,1,2,2,...,,,2,,,3,2,,1,3


In [31]:
percepcion_df.columns

Index(['Año', 'SEXO', 'EDAD', 'NOM_MUN', 'AP4_2_03', 'AP4_2_05', 'AP4_3_1',
       'AP4_3_2', 'AP4_3_3', 'AP4_4_03', 'AP4_4_12', 'AP4_5_09', 'AP4_5_10',
       'AP4_5_11', 'AP4_5_13', 'AP4_5_14', 'AP4_10_01', 'AP4_10_02',
       'AP4_10_09', 'AP5_4_01', 'AP5_4_02', 'AP5_4_03', 'AP5_4_04', 'AP5_4_05',
       'AP5_4_06', 'AP5_4_07', 'AP5_4_08', 'AP5_4_09', 'AP5_4_10', 'ESTRATO',
       'AP4_4_A'],
      dtype='object')

In [32]:
percepcion_df.rename(columns={
    'SEXO':'Sexo',
    'EDAD':'Edad',
    'NOM_MUN':'Municipio',
    'AP4_2_03':'Pr Narcotrafico',
    'AP4_2_05':'Pr Inseguridad',
    'AP4_3_1':'P Seg Colonia',
    'AP4_3_2':'P Seg Municipio',
    'AP4_3_3':'P Seg Estado',
    'AP4_4_03':'P Seg Calle',
    'AP4_4_12':'P Seg Parque',
    'AP4_5_09':'P Robos Frecuentes',
    'AP4_5_10':'P Venta Droga',
    'AP4_5_11':'P Disparos Frecuentes',
    'AP4_5_13':'P Secuestros',
    'AP4_5_14':'P Homicidios',
    'AP4_10_01':'D Salir Noche',
    'AP4_10_02': 'D Salir Menores',
    'AP4_10_09':'D Salir Caminar',
    'AP5_4_01':'C Transito Mun',
    'AP5_4_02':'C Policia Mun',
    'AP5_4_03':'C Policia Est',
    'AP5_4_04':'C Guardia Na',
    'AP5_4_05':'C Policia Min',
    'AP5_4_06':'C Ministerio Pub',
    'AP5_4_07':'C Fiscalia Gral Rep',
    'AP5_4_08':'C Ejercito',
    'AP5_4_09':'C Marina',
    'AP5_4_10':'C Jueces',
    'ESTRATO':'Estrato',
    'AP4_4_A':'P Seg Caminar Solo Noche'
}, inplace=True)
percepcion_df

,Año,Sexo,Edad,Municipio,Pr Narcotrafico,Pr Inseguridad,P Seg Colonia,P Seg Municipio,P Seg Estado,P Seg Calle,...,C Policia Est,C Guardia Na,C Policia Min,C Ministerio Pub,C Fiscalia Gral Rep,C Ejercito,C Marina,C Jueces,Estrato,P Seg Caminar Solo Noche
72941,2017,2,25,Cajeme,0,1,1,2,2,2,...,4,4,4,4,4,2,2,4,3,0
72942,2017,1,43,Cajeme,0,1,1,1,1,2,...,3,3,,2,,2,1,2,3,0
72943,2017,2,51,Cajeme,0,1,2,2,2,2,...,3,3,,,,2,2,,3,0
72944,2017,1,30,Cajeme,0,1,2,2,2,2,...,2,2,,,2,2,1,,3,0
72945,2017,2,81,Cajeme,0,1,2,2,1,2,...,,,,,,,,,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532747,2022,1,80,Yécora,1,0,1,1,9,3,...,,,,,,,,,1,5
532748,2022,1,62,Yécora,0,0,1,1,1,1,...,2,2,3,2,2,1,2,2,1,1
532749,2022,2,53,Yécora,1,1,2,2,2,1,...,2,,2,,,2,2,,1,5
532750,2022,2,53,Yécora,0,1,2,1,2,2,...,,,2,,,3,2,,1,3


In [33]:
percepcion_df['C Transito Mun'] = percepcion_df['C Transito Mun'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Transito Mun'] = percepcion_df['C Transito Mun'].astype(int)

percepcion_df['C Policia Mun'] = percepcion_df['C Policia Mun'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Policia Mun'] = percepcion_df['C Policia Mun'].astype(int)

percepcion_df['C Policia Est'] = percepcion_df['C Policia Est'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Policia Est'] = percepcion_df['C Policia Est'].astype(int)

percepcion_df['C Guardia Na'] = percepcion_df['C Guardia Na'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Guardia Na'] = percepcion_df['C Guardia Na'].astype(int)

percepcion_df['C Policia Min'] = percepcion_df['C Policia Min'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Policia Min'] = percepcion_df['C Policia Min'].astype(int)

percepcion_df['C Ministerio Pub'] = percepcion_df['C Ministerio Pub'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Ministerio Pub'] = percepcion_df['C Ministerio Pub'].astype(int)

percepcion_df['C Fiscalia Gral Rep'] = percepcion_df['C Fiscalia Gral Rep'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Fiscalia Gral Rep'] = percepcion_df['C Fiscalia Gral Rep'].astype(int)

percepcion_df['C Ejercito'] = percepcion_df['C Ejercito'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Ejercito'] = percepcion_df['C Ejercito'].astype(int)

percepcion_df['C Marina'] = percepcion_df['C Marina'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Marina'] = percepcion_df['C Marina'].astype(int)

percepcion_df['C Jueces'] = percepcion_df['C Jueces'].replace(['NA', ''], '0', regex=True)
percepcion_df['C Jueces'] = percepcion_df['C Jueces'].astype(int)

In [34]:
# Insertamos una nueva columna para resumir si la población tiene preocupación 
# en alguno de los temas considerados como relevantes para el estudio.
percepcion_df['Tiene Preocupacion'] = (
    (percepcion_df['Pr Narcotrafico'] == 1) |
    (percepcion_df['Pr Inseguridad'] == 1)
)

percepcion_df['Colonia Insegura'] = (
    (percepcion_df['P Seg Colonia'] == 2) |
    (percepcion_df['P Seg Calle'] == 2) |
    (percepcion_df['P Seg Parque'] == 2)
)

percepcion_df['Municipio Inseguro'] = (
    (percepcion_df['P Seg Municipio'] == 2) |
    (percepcion_df['P Robos Frecuentes'] == 1) |
    (percepcion_df['P Venta Droga'] == 1) |
    (percepcion_df['P Disparos Frecuentes'] == 1) |
    (percepcion_df['P Secuestros'] == 1) |
    (percepcion_df['P Homicidios'] == 1)
)
        
percepcion_df['Estado Inseguro'] = (percepcion_df['P Seg Estado'] == 2)
    
percepcion_df['Afectacion Forma de Vida'] = (
    (percepcion_df['D Salir Noche'] == 1) |
    (percepcion_df['D Salir Caminar'] == 1) |
    (percepcion_df['D Salir Menores'] == 1)
)
    
percepcion_df['Confianza en Transito'] = (
    (percepcion_df['C Transito Mun'] == 1) | 
    (percepcion_df['C Transito Mun'] == 2)
)

percepcion_df['Confianza en Policia Mun'] = (
    (percepcion_df['C Policia Mun'] == 1) | 
    (percepcion_df['C Policia Mun'] == 2)
)
    
percepcion_df['Confianza en Policia Est'] = (
    (percepcion_df['C Policia Est'] == 1) | 
    (percepcion_df['C Policia Est'] == 2)
)
    
percepcion_df['Confianza en Guardia Na'] = (
    (percepcion_df['C Guardia Na'] == 1) | 
    (percepcion_df['C Guardia Na'] == 2)
)

percepcion_df['Confianza en Policia Min'] = (
    (percepcion_df['C Policia Min'] == 1) | 
    (percepcion_df['C Policia Min'] == 2)
)

percepcion_df['Confianza en Ministerio Pub'] = (
    (percepcion_df['C Ministerio Pub'] == 1) | 
    (percepcion_df['C Ministerio Pub'] == 2)
)

percepcion_df['Confianza en Fiscalia G R'] = (
    (percepcion_df['C Fiscalia Gral Rep'] == 1) | 
    (percepcion_df['C Fiscalia Gral Rep'] == 2)
)

percepcion_df['Confianza en Ejercito'] = (
    (percepcion_df['C Ejercito'] == 1) | 
    (percepcion_df['C Ejercito'] == 2)
)

percepcion_df['Confianza en Marina'] = (
    (percepcion_df['C Marina'] == 1) | 
    (percepcion_df['C Marina'] == 2)
)

percepcion_df['Confianza en Jueces'] = (
    (percepcion_df['C Jueces'] == 1) | 
    (percepcion_df['C Jueces'] == 2)
)

In [35]:
percepcion_df['C Policia Mun'].unique()

array([3, 2, 0, 4, 9, 1])

In [36]:
percepcion_df = percepcion_df[['Año', 'Municipio', 'Tiene Preocupacion',
                        'Colonia Insegura', 'Municipio Inseguro', 'Estado Inseguro',
                        'Afectacion Forma de Vida', 'Confianza en Transito', 
                        'Confianza en Policia Mun', 'Confianza en Policia Est',
                        'Confianza en Guardia Na', 'Confianza en Policia Min', 
                        'Confianza en Ministerio Pub', 'Confianza en Fiscalia G R',
                        'Confianza en Ejercito', 'Confianza en Marina', 'Confianza en Jueces']]
percepcion_df

,Año,Municipio,Tiene Preocupacion,Colonia Insegura,Municipio Inseguro,Estado Inseguro,Afectacion Forma de Vida,Confianza en Transito,Confianza en Policia Mun,Confianza en Policia Est,Confianza en Guardia Na,Confianza en Policia Min,Confianza en Ministerio Pub,Confianza en Fiscalia G R,Confianza en Ejercito,Confianza en Marina,Confianza en Jueces
72941,2017,Cajeme,True,True,True,True,True,False,False,False,False,False,False,False,True,True,False
72942,2017,Cajeme,True,True,True,False,False,True,True,False,False,False,True,False,True,True,True
72943,2017,Cajeme,True,True,True,True,False,True,True,False,False,False,False,False,True,True,False
72944,2017,Cajeme,True,True,True,True,True,True,True,True,True,False,False,True,True,True,False
72945,2017,Cajeme,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532747,2022,Yécora,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
532748,2022,Yécora,False,False,True,False,False,True,True,True,True,False,True,True,True,True,True
532749,2022,Yécora,True,True,True,True,True,False,True,True,False,True,False,False,True,True,False
532750,2022,Yécora,True,True,True,True,True,False,False,False,False,True,False,False,False,True,False


In [37]:
percepcion_df['Confianza en Transito'].unique()

array([False,  True])

In [38]:
percepcion_df = percepcion_df.groupby(['Año', 'Municipio']).sum() / percepcion_df.groupby(['Año', 'Municipio']).count()

In [39]:
percepcion_df

Tiene Preocupacion  Colonia Insegura  \
Año  Municipio                                                     
2017 Agua Prieta                      0.500000          0.500000   
     Altar                            0.647059          0.411765   
     Benito Juárez                    0.750000          0.650000   
     Bácum                            0.857143          0.666667   
     Caborca                          0.666667          0.694444   
...                                        ...               ...   
2022 Navojoa                          0.464789          0.422535   
     Nogales                          0.719178          0.684932   
     San Luis Río Colorado            0.764706          0.852941   
     Santa Ana                        0.740741          0.537037   
     Yécora                           0.562500          0.687500   

                            Municipio Inseguro  Estado Inseguro  \
Año  Municipio                                                    
2017 Agua Prieta                      0.666667         0.537037   
     Altar                            0.588235         0.529412   
     Benito Juárez                    1.000000         0.500000   
     Bácum                            0.904762         0.523810   
     Caborca                          0.805556         0.527778   
...                                        ...              ...   
2022 Navojoa                          0.394366         0.690141   
     Nogales                          0.910959         0.842466   
     San Luis Río Colorado            0.926471         0.838235   
     Santa Ana                        0.722222         0.833333   
     Yécora                           0.875000         0.625000   

                            Afectacion Forma de Vida  Confianza en Transito  \
Año  Municipio                                                                
2017 Agua Prieta                            0.481481               0.388889   
     Altar                                  0.529412               0.529412   
     Benito Juárez                          0.850000               0.300000   
     Bácum                                  0.666667               0.190476   
     Caborca                                0.611111               0.333333   
...                                              ...                    ...   
2022 Navojoa                                0.267606               0.535211   
     Nogales                                0.609589               0.369863   
     San Luis Río Colorado                  0.772059               0.279412   
     Santa Ana                              0.796296               0.388889   
     Yécora                                 0.687500               0.125000   

                            Confianza en Policia Mun  \
Año  Municipio                                         
2017 Agua Prieta                            0.500000   
     Altar                                  0.882353   
     Benito Juárez                          0.400000   
     Bácum                                  0.523810   
     Caborca                                0.305556   
...                                              ...   
2022 Navojoa                                0.535211   
     Nogales                                0.417808   
     San Luis Río Colorado                  0.411765   
     Santa Ana                              0.388889   
     Yécora                                 0.375000   

                            Confianza en Policia Est  Confianza en Guardia Na  \
Año  Municipio                                                                  
2017 Agua Prieta                            0.444444                 0.500000   
     Altar                                  0.647059                 0.588235   
     Benito Juárez                          0.550000                 0.500000   
     Bácum                                  0.619048                 0.476190   
     Caborca                

In [40]:
# Guardamos el Data Frame 'percepcion_df' de pandas como .parquet
percepcion_df.to_parquet(subdir + 'percepcion_df.parquet')

# Información sobre la Densidad Poblacional en Sonora
Revisamos el dataset del INEGI para 2020 con la información de Población Total por Municipio para Normalizar las incidencias delictivas. Para el caso de estudio se asumirá que la población se mantiene estable a lo largo de los 6 años que comprenden los datos.

In [41]:
poblacion_df

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,26,Sonora,0,Total de la entidad Sonora,0,Total de la Entidad,NaN,NaN,NaN,2944840,...,811378,530511,501521,210780,126299,32807,48777,302080,11895,*
1,26,Sonora,0,Total de la entidad Sonora,9998,Localidades de una vivienda,NaN,NaN,NaN,16883,...,3776,686,1563,139,92,629,627,3598,261,*
2,26,Sonora,0,Total de la entidad Sonora,9999,Localidades de dos viviendas,NaN,NaN,NaN,5087,...,1322,268,639,44,30,217,222,1206,100,*
3,26,Sonora,1,Aconchi,0,Total del Municipio,NaN,NaN,NaN,2563,...,704,254,613,53,43,35,61,468,18,*
4,26,Sonora,1,Aconchi,1,Aconchi,"110°13'42.787"" W","29°49'33.141"" N",607.0,1650,...,473,193,394,42,34,19,32,290,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,26,Sonora,72,San Ignacio Río Muerto,340,Ninguno [José Palma],"110°11'56.440"" W","27°20'44.190"" N",8.0,2,...,*,*,*,*,*,*,*,*,*,1
7496,26,Sonora,72,San Ignacio Río Muerto,342,Cornelio Saucedo,"110°21'43.366"" W","27°23'25.691"" N",7.0,2,...,*,*,*,*,*,*,*,*,*,1
7497,26,Sonora,72,San Ignacio Río Muerto,343,Santa Bárbara,"110°12'33.900"" W","27°19'01.253"" N",6.0,1,...,*,*,*,*,*,*,*,*,*,1
7498,26,Sonora,72,San Ignacio Río Muerto,9998,Localidades de una vivienda,NaN,NaN,NaN,296,...,85,11,29,2,1,19,15,86,5,*


In [42]:
poblacion_df.drop(poblacion_df[poblacion_df['LOC'] != 0].index, inplace = True)
poblacion_df

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,26,Sonora,0,Total de la entidad Sonora,0,Total de la Entidad,NaN,NaN,NaN,2944840,...,811378,530511,501521,210780,126299,32807,48777,302080,11895,*
3,26,Sonora,1,Aconchi,0,Total del Municipio,NaN,NaN,NaN,2563,...,704,254,613,53,43,35,61,468,18,*
13,26,Sonora,2,Agua Prieta,0,Total del Municipio,NaN,NaN,NaN,91929,...,25677,18734,19940,6616,4143,822,965,7320,173,*
114,26,Sonora,3,Alamos,0,Total del Municipio,NaN,NaN,NaN,24976,...,5763,1409,4292,461,232,904,1248,5304,527,*
392,26,Sonora,4,Altar,0,Total del Municipio,NaN,NaN,NaN,9492,...,2480,1275,1931,368,275,118,143,1221,39,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,26,Sonora,68,Villa Pesqueira,0,Total del Municipio,NaN,NaN,NaN,1043,...,322,88,273,19,11,36,44,274,13,*
7143,26,Sonora,69,Yécora,0,Total del Municipio,NaN,NaN,NaN,4793,...,1150,299,908,36,35,242,297,1155,125,*
7216,26,Sonora,70,General Plutarco Elías Calles,0,Total del Municipio,NaN,NaN,NaN,13627,...,3804,1952,3042,630,460,215,193,1860,58,*
7291,26,Sonora,71,Benito Juárez,0,Total del Municipio,NaN,NaN,NaN,21692,...,5230,1942,2433,363,195,365,682,3737,189,*


In [43]:
poblacion_df = poblacion_df[['NOM_MUN', 'POBTOT']]

In [44]:
poblacion_df.rename(columns={'NOM_MUN':'Municipio', 
                             'POBTOT':'Poblacion'}, 
                    inplace=True)
poblacion_df

C:\Users\oaiza\AppData\Local\Temp\ipykernel_10152\4158863762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poblacion_df.rename(columns={'NOM_MUN':'Municipio',


,Municipio,Poblacion
0,Total de la entidad Sonora,2944840
3,Aconchi,2563
13,Agua Prieta,91929
114,Alamos,24976
392,Altar,9492
...,...,...
7105,Villa Pesqueira,1043
7143,Yécora,4793
7216,General Plutarco Elías Calles,13627
7291,Benito Juárez,21692


In [45]:
poblacion_df.drop(poblacion_df[poblacion_df['Municipio'] == 'Total de la entidad Sonora'].index, inplace = True)

C:\Users\oaiza\AppData\Local\Temp\ipykernel_10152\3439326526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poblacion_df.drop(poblacion_df[poblacion_df['Municipio'] == 'Total de la entidad Sonora'].index, inplace = True)


Hacemos merge de los DataFrames de `delitos_df` y `poblacion_df` para normalizar las observaciones delictivas per capita.

In [46]:
delitos_df = pd.merge(delitos_df, poblacion_df, how='outer', on='Municipio')

In [47]:
delitos_df['Abuso sexual'] = delitos_df['Abuso sexual'] / delitos_df['Poblacion']
delitos_df['Feminicidio'] = delitos_df['Feminicidio'] / delitos_df['Poblacion']
delitos_df['Homicidio'] = delitos_df['Homicidio'] / delitos_df['Poblacion']
delitos_df['Narcomenudeo'] = delitos_df['Narcomenudeo'] / delitos_df['Poblacion']
delitos_df['Robo'] = delitos_df['Robo'] / delitos_df['Poblacion']
delitos_df['Secuestro'] = delitos_df['Secuestro'] / delitos_df['Poblacion']

In [48]:
delitos_df

,Año,Municipio,Abuso sexual,Feminicidio,Homicidio,Narcomenudeo,Robo,Secuestro,Poblacion
0,2017,Aconchi,0.00000,0.000000,0.000780,0.000390,0.002731,0.0,2563
1,2018,Aconchi,0.00000,0.000000,0.000000,0.000390,0.001951,0.0,2563
2,2019,Aconchi,0.00000,0.000000,0.000000,0.000000,0.000780,0.0,2563
3,2020,Aconchi,0.00000,0.000000,0.000390,0.000000,0.000000,0.0,2563
4,2021,Aconchi,0.00039,0.000000,0.000000,0.000000,0.001561,0.0,2563
...,...,...,...,...,...,...,...,...,...
427,2018,Yécora,0.00000,0.000209,0.003755,0.000209,0.001669,0.0,4793
428,2019,Yécora,0.00000,0.000000,0.000626,0.000000,0.002295,0.0,4793
429,2020,Yécora,0.00000,0.000000,0.002504,0.000000,0.005007,0.0,4793
430,2021,Yécora,0.00000,0.000000,0.001460,0.000000,0.010641,0.0,4793


Se realiza un merge de los DataFrames de incidencias delictivas `delitos_df` con datos normalizados por tamaño de población con el de Persepción de la Inseguridad `percepcion_df` en el DataFrame `seguridad_df` y se guarda como .parquet para su manejo posterior.

In [52]:
seguridad_df = pd.merge(delitos_df, percepcion_df, how='outer', on=['Año', 'Municipio'])
seguridad_df

,Año,Municipio,Abuso sexual,Feminicidio,Homicidio,Narcomenudeo,Robo,Secuestro,Poblacion,Tiene Preocupacion,...,Confianza en Transito,Confianza en Policia Mun,Confianza en Policia Est,Confianza en Guardia Na,Confianza en Policia Min,Confianza en Ministerio Pub,Confianza en Fiscalia G R,Confianza en Ejercito,Confianza en Marina,Confianza en Jueces
0,2017,Aconchi,0.00000,0.0,0.000780,0.00039,0.002731,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,Aconchi,0.00000,0.0,0.000000,0.00039,0.001951,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Aconchi,0.00000,0.0,0.000000,0.00000,0.000780,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,Aconchi,0.00000,0.0,0.000390,0.00000,0.000000,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Aconchi,0.00039,0.0,0.000000,0.00000,0.001561,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,2019,Yécora,0.00000,0.0,0.000626,0.00000,0.002295,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,2020,Yécora,0.00000,0.0,0.002504,0.00000,0.005007,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,2021,Yécora,0.00000,0.0,0.001460,0.00000,0.010641,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,2022,Yécora,0.00000,0.0,0.000209,0.00000,0.003547,0.0,4793.0,0.562500,...,0.125,0.375000,0.437500,0.562500,0.3125,0.187500,0.312500,0.812500,0.750000,0.187500


In [53]:
for col in seguridad_df.columns:
    seguridad_df[col].fillna(pd.NA)
seguridad_df

,Año,Municipio,Abuso sexual,Feminicidio,Homicidio,Narcomenudeo,Robo,Secuestro,Poblacion,Tiene Preocupacion,...,Confianza en Transito,Confianza en Policia Mun,Confianza en Policia Est,Confianza en Guardia Na,Confianza en Policia Min,Confianza en Ministerio Pub,Confianza en Fiscalia G R,Confianza en Ejercito,Confianza en Marina,Confianza en Jueces
0,2017,Aconchi,0.00000,0.0,0.000780,0.00039,0.002731,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,Aconchi,0.00000,0.0,0.000000,0.00039,0.001951,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Aconchi,0.00000,0.0,0.000000,0.00000,0.000780,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,Aconchi,0.00000,0.0,0.000390,0.00000,0.000000,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Aconchi,0.00039,0.0,0.000000,0.00000,0.001561,0.0,2563.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,2019,Yécora,0.00000,0.0,0.000626,0.00000,0.002295,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,2020,Yécora,0.00000,0.0,0.002504,0.00000,0.005007,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,2021,Yécora,0.00000,0.0,0.001460,0.00000,0.010641,0.0,4793.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,2022,Yécora,0.00000,0.0,0.000209,0.00000,0.003547,0.0,4793.0,0.562500,...,0.125,0.375000,0.437500,0.562500,0.3125,0.187500,0.312500,0.812500,0.750000,0.187500


In [54]:
# Guardamos el Data Frame 'delitos_df' de pandas como .parquet
seguridad_df.to_parquet(subdir + 'seguridad_df.parquet')